This notebook script reads LLM statistics from CSV files, processes the data, and generates a summary DataFrame.

> **⚠️ This script expect specific CSV file formats**<br>
> Use the `scripts/engine_validator/validator.py` to generate the CSV files or create your own CSV files with the same format:
>   
>   - columns: 
>       - uc_type: (str) Use-case type of the prompt
>           (e.g: general, research, sNw, query, ...)
>       - prompt: (str) The used prompt for the LLM.
>       - llm_specifier: (str) Concatenated form of used LLM models (1: analyzer and synthesizer, 2: data service)
>           (e.g: claude_3_haiku-claude_3_haiku, llama3_8-claude_3_haiku)
>       - content_length: (int) The length of the LLM response content in characters.
>       - processing_time: (float) The processing time of the LLM response in seconds.            
>   - delimiter: \x1f (ASCII unit separator) - <br>
>   - example:
>       - header: `uc_typepromptllm_specifiercontent_lengthprocessing_time`
>       - rows: `generalHow are you?llm_model_1-llm_model_212.34`

It compares the performance of different LLMs across various prompts and environments.
It calculates averages, differences, and formats the output for better readability.
It also exports the final summary to an Excel file.

---

ℹ️ It is designed to help analyze the performance of different LLMs across various prompts and environments.
It also can be used to compare your local changes against the default dev/staging/production environment.

In [ ]:
# ====================   Notebook run-configuration   ====================
# ⚠️ Set the filenames and environments for the CSV files to read!
csv_files = {
    "dev": "dev_llm_stats.csv",  # "dev" is the default environment
    "feat": "custom_llm_stats.csv",  # "feat" is the feature or custom environment
}

# ⚠️ Set the number of decimal places for processing times
DECIMAL_PLACES = 3

# ⚠️ Set the analysis grouping values
grouping_columns = {
    "uc_type": ["uc_type", "env"],  # Generate UC_TYPE based comparison
    "prompt": ["prompt", "env"]  # Generate PROMPT based comparison
}

GROUPING_TYPE = "prompt"  # Options: "uc_type" or "prompt"
GROUPING_COLUMNS = grouping_columns[GROUPING_TYPE]

In [ ]:
import pandas as pd
import os
from scripts.engine_validator.prompts import UCType

In [ ]:
# ==================== Helper functions ====================
def round_values(x, decimal_places=2):
    """Round float values in list or single values"""
    if isinstance(x, str):
        return x
    if isinstance(x, list):
        return [round(val, decimal_places) for val in x if isinstance(val, (int, float))]  # Handle non-numeric values gracefully
    return round(x, decimal_places) if pd.notnull(x) else x


def unflatten_columns(df, separator="_", preserve_cols=None):
    """Convert flattened column names back to multi-level columns, except preserve_cols that will be kept as single-level columns"""
    new_columns = []
    for col in df.columns:
        if col in preserve_cols:
            new_columns.append((col, ""))
        else:
            parts = col.split(separator, 1)  # Split only on first separator
            if len(parts) > 1:
                new_columns.append((parts[1], parts[0]))  # Reverse the order back to original
            else:
                new_columns.append((parts[0], ""))

    df.columns = pd.MultiIndex.from_tuples(new_columns)
    return df

In [ ]:
# ====================  Load and combine CSV files  ====================
LOCAL_FOLDER = os.path.abspath("")

# PRE-VALIDATION, ensure the required CSV files exist
for prefix, filename in csv_files.items():
    if not os.path.exists(filename):
        raise FileNotFoundError(f"File {filename} does not exist in the folder: {LOCAL_FOLDER}.")

prefixes = list(csv_files.keys())
dev, feat = prefixes
selected_cols = ["uc_type", "prompt", "llm_specifier", "content_length", "processing_time"]

# Read the CSV files and select the required columns
dfs = []
for pfx, filename in csv_files.items():
    df = pd.read_csv(filename, delimiter="\x1f")
    df = df[selected_cols]
    df["env"] = pfx  # This adds the source identifier
    dfs.append(df)

# Combine vertically (stack all rows)
combined_df = pd.concat(dfs, ignore_index=True)
combined_df.head(1)

In [ ]:
# ====================  Data aggregation and statistics  ====================
# Ensure processing_time is float
combined_df["processing_time"] = pd.to_numeric(combined_df["processing_time"], errors="coerce")

# Check if there are multiple llm_specifier values in the data, if yes then auto-fix GROUPING_COLUMNS
llm_specifiers = combined_df["llm_specifier"].unique()
if len(llm_specifiers) > 1:
    print(f"⚠️ Multiple llm_specifier values detected: {llm_specifiers}\nAdjust GROUPING_COLUMNS accordingly.")
    if "llm_specifier" not in GROUPING_COLUMNS:
        GROUPING_COLUMNS.append("llm_specifier")


# Group by based on the pre-configured columns then aggregate
if GROUPING_TYPE == "uc_type":
    grouping_extras= {
        # "uc_type": "first",
        "prompt": list,
    }
else:
    grouping_extras = {
        "uc_type": "first",
    }


grouped = (
    combined_df.groupby(GROUPING_COLUMNS)
    .agg(
        {
            **grouping_extras,
            "content_length": list,
            "processing_time": [list, "mean", "median", "std"],
        }
    )
    .reset_index()
)

# Renaming columns
print(grouped.columns)
grouped.columns = [
    *GROUPING_COLUMNS,
    *grouping_extras.keys(),
    "content_lengths",
    "processing_times",
    "processing_avg",
    "processing_median",
    "processing_std",
]

# Pivot to get dev and custom as separate columns
agg_df = grouped.pivot(index=GROUPING_COLUMNS[0], columns=GROUPING_COLUMNS[1:]).reset_index()
agg_df.columns = [f"{col[1]}_{col[0]}" if col[1] else col[0] for col in agg_df.columns]

# Flatten the multi-level columns and keep only one uc_type column
uc_type_cols = [col for col in agg_df.columns if "uc_type" in col]
agg_df = agg_df.rename(columns={uc_type_cols.pop(0): "uc_type"})
for col in uc_type_cols:
    agg_df = agg_df.drop(columns=[col])

agg_df.head(1)

In [ ]:
# ====================  Post-processing and refinement  ====================
# UC_TYPE translation
agg_df["uc_type"] = agg_df["uc_type"].apply(lambda x: UCType(x).value)

# Refine column names
stat_df = agg_df.rename(columns=lambda x: x.replace("prompt", "prompts") if "prompt" in x else x)

# Calculate comparison columns for env specific aggregates
stat_df["diff_processing_avg"] = stat_df[f"{feat}_processing_avg"] / stat_df[f"{dev}_processing_avg"] * 100
stat_df["diff_processing_median"] = stat_df[f"{feat}_processing_median"] / stat_df[f"{dev}_processing_median"] * 100
stat_df["diff_processing_std"] = stat_df[f"{feat}_processing_std"] / stat_df[f"{dev}_processing_std"] * 100

# Round float values in processing time columns
processing_cols = [col for col in stat_df.columns if "processing" in col]
for col in processing_cols:
    stat_df[col] = stat_df[col].apply(lambda x: round_values(x, DECIMAL_PLACES))

stat_df.head(3)

In [ ]:
# ====================  Summarization and post formatting  ====================
# Calculate total averages for numeric columns
if GROUPING_TYPE == "uc_type":
    prompts_columns = {
        f"{dev}_prompts": "",
        f"{feat}_prompts": "",
    }
else:
    prompts_columns = {
        "prompts": ""
    }
total_avg_row = {
    "uc_type": "TOTAL AVERAGE",
    **prompts_columns,
    f"{dev}_content_lengths": "",
    f"{feat}_content_lengths": "",
    f"{dev}_processing_times": "",
    f"{feat}_processing_times": "",
    f"{dev}_processing_avg": stat_df[f"{dev}_processing_avg"].mean(),
    f"{feat}_processing_avg": stat_df[f"{feat}_processing_avg"].mean(),
    "diff_processing_avg": stat_df["diff_processing_avg"].mean(),
    f"{dev}_processing_median": stat_df[f"{dev}_processing_median"].mean(),
    f"{feat}_processing_median": stat_df[f"{feat}_processing_median"].mean(),
    "diff_processing_median": stat_df["diff_processing_median"].mean(),
    f"{dev}_processing_std": stat_df[f"{dev}_processing_std"].mean(),
    f"{feat}_processing_std": stat_df[f"{feat}_processing_std"].mean(),
    "diff_processing_std": stat_df["diff_processing_std"].mean(),
}

# Add the total row to the dataframe
output_df = pd.concat([stat_df, pd.DataFrame([total_avg_row])], ignore_index=True)
processing_cols = [col for col in output_df.columns if "processing" in col]
for col in processing_cols:
    output_df[col] = output_df[col].apply(lambda x: round_values(x, DECIMAL_PLACES))

# Reorder columns
output_df = output_df[[*total_avg_row.keys()]]

# Convert flat columns to multi-level columns
single_lvl_columns = ["uc_type"]
output_df = unflatten_columns(output_df, separator="_", preserve_cols=single_lvl_columns)

# Drop columns where all values are NaN
output_df = output_df.dropna(axis=1, how="all")
output_df.head(3)

In [ ]:
# End result
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  # Set a wider display width
pd.set_option('display.max_colwidth', None)  # Show full length of Prompt column
output_df

In [ ]:
# Export stat_df to an Excel file in LOCAL_FOLDER
if GROUPING_TYPE == "uc_type":
    output_file = "uc_type_stats_summary.xlsx"
else:
    output_file = "prompt_stats_summary.xlsx"

output_path = os.path.join(LOCAL_FOLDER, output_file)
output_df.to_excel(output_path, index=True)
print(f"Exported to {output_path}")